#### Predicting wages with a Keras neural network

In this analysis I'll build a basic nueral network model in Keras. One of the goals with this analysis is to quickly be able to run more complex neural network models on larger datasets. Speaking of datasets, I'm using one that should allow for the prediction of an individuals hourly wages given characteristics like their industry, education and level of experience. Unfortunately, this dataset is (seemingly) no longer hosted anywhere notable, so I'll make it available on my [personal github page](https://github.com/brukeg/notebooks/tree/master/datasets/predicting-wages.csv).

In [15]:
# Import necessary modules
import pandas as pd
import numpy as np
import keras

from sklearn.model_selection import train_test_split
from keras.layers import Dense
from keras.models import Sequential

In [17]:
# import and explore the data set
df = pd.read_csv('datasets/hourly_wages.csv')


# create arrays for the features and target variable
target = df.wage_per_hour.values
predictors = df.drop('wage_per_hour', axis=1).values

X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=42)

df.head()

,wage_per_hour,union,education_yrs,experience_yrs,age,female,marr,south,manufacturing,construction
0,5.10,0,8,21,35,1,1,0,1,0
1,4.95,0,9,42,57,1,1,0,1,0
2,6.67,0,12,1,19,0,0,0,1,0
3,4.00,0,12,4,22,0,0,0,0,0
4,7.50,0,12,17,35,0,1,0,0,0


#### Getting Started 
<p>To start off with, I'll take a skeleton of a neural network and add a hidden layer and an output layer. As refresher, a nueral network contains an input layer, at least 1 hidden layer, and an output layer. I'll then fit that model and see Keras do the optimization so the model continually gets better.</p>

<p><img src="datasets/Hidden_Layer_print.png" width="50%" align="center"></p>

In [7]:
# Save the number of columns in predictors
n_cols = predictors.shape[1]

# Set up the model
model = Sequential()

# Add the first layer (input)
model.add(Dense(50, activation='relu', input_shape=(n_cols,)))

# Add the second layer (hidden)
model.add(Dense(32, activation='relu'))

# Add the output layer
model.add(Dense(1))

#### Compiling and Fitting the Model
I'm now going to compile the model I specified above. To compile the model, I'll simply need to specify the optimizer and loss function to use. The Adam optimizer is an excellent first choice, and MSE will work just fine for my purposes as a loss function. You can do further reading about it and other keras optimizers [here](https://keras.io/optimizers/#adam), and if you are really curious to learn more, you can read the [original paper](https://arxiv.org/abs/1412.6980v8) that introduced the Adam optimizer.

Finally, I'll fit the model using the `.fit()` method.

In [12]:
# Specify the model
n_cols = predictors.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Verify that model contains information from compiling
print("Loss function: " + model.loss)

Loss function: mean_squared_error


In [18]:
# Specify the model
n_cols = predictors.shape[1]
model = Sequential()
model.add(Dense(50, activation='relu', input_shape = (n_cols,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model.fit(X_train, y_train)

Epoch 1/1
373/373 [==============================] - 1s 3ms/step - loss: 30.7995


In [33]:
# Calculate predictions: predictions
predictions = model.predict(X_test)

# Calculate predicted wages
predicted_prob_true = predictions[:,0]

# print predicted_prob_true
print(predicted_prob_true)

# print the differnce between known wages and our predicted ones
#print(predicted_prob_true-target)

[14.642925   5.0396843  6.522493   9.660091   6.7229195  6.852784
 10.493897   5.3469477  7.998204   7.487485   8.494655   6.9070754
  7.70672    7.625542   7.20399    6.53131    7.934996  13.16728
  8.4557    10.16424    8.951924  14.493992   7.0996046  8.762305
  4.39365    9.08064   10.469973  11.224567  12.831081  12.685733
  6.773531  12.028591   7.1843157  9.966084  13.454934   5.0752106
  8.015115   8.928014   7.341394   7.520835  12.755751   8.796549
  8.41497   12.734032  12.572275  14.453107   6.8306456  9.7016735
  6.602378   5.711293   8.467966  13.168735  11.36424   11.987953
  7.6612186  4.8784084  9.672422   7.919685  10.447189   6.9081707
  6.4492254 10.559987   7.3353686  9.048932   6.284211   8.777804
 10.559987   8.486797  10.58222    6.489592   4.7867994  8.265329
 13.246117   4.7984037  5.220437  13.383447  12.034864   4.6860037
 13.649389   5.473402  13.9430685  5.055064   8.58242    9.447361
  6.561168   6.8306456  6.1303015  7.27864    7.70672    5.9459987
  6.0

In [ ]:
"""FROM TITANIC EXERCISE"""

# Specify, compile, and fit the model
model = Sequential()
model.add(Dense(32, activation='relu', input_shape = (n_cols,)))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer='sgd', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
model.fit(predictors, target)

# Calculate predictions: predictions
predictions = model.predict(pred_data)

# Calculate predicted probability of survival: predicted_prob_true
predicted_prob_true = predictions[:,1]

# print predicted_prob_true
print(predicted_prob_true)

In [ ]:
"""FROM TITANIC EXERCISE"""

# Import the SGD optimizer
from keras.optimizers import SGD

# Create list of learning rates: lr_to_test
lr_to_test = [.000001, 0.01, 1]

# Loop over learning rates
for lr in lr_to_test:
    print('\n\nTesting model with learning rate: %f\n'%lr )
    
    # Build new model to test, unaffected by previous models
    model = get_new_model()
    
    # Create SGD optimizer with specified learning rate: my_optimizer
    my_optimizer = SGD(lr=lr)
    
    # Compile the model
    model.compile(optimizer=my_optimizer, loss='categorical_crossentropy' )
    
    # Fit the model
    model.fit(predictors, target)
    